In [1]:
import os

os.environ['KERAS_BACKEND'] = "torch"

## Kickstarter Less Dataset (256k)


In [2]:
from train import AbaeModelConfiguration
from core.train import AbaeModelManager

config = AbaeModelConfiguration(
    corpus_file="../data/corpus.preprocessed.kickstarter_removed.256k.csv",
    model_name="abae.kickstarter_removed.256k", aspect_size=16, max_vocab_size=40000,
)

manager = AbaeModelManager(config)

/home/jacopo/PycharmProjects/nlp-course-project/venv/lib/python3.12/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
INFO:gensim.utils:loading Word2Vec object from output/abae.kickstarter_removed.256k.embeddings.model
DEBUG:smart_open.smart_open_lib:{'uri': 'output/abae.kickstarter_removed.256k.embeddings.model', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
INFO:gensim.utils:loading wv recursively from output/abae.kickstarter_removed.256k.embeddings.model.wv.* with mmap=None
INFO:gensim.utils:setting ignored attribute cum_table to None
INFO

In [3]:
inference_model = manager.prepare_evaluation_model()

/home/jacopo/PycharmProjects/nlp-course-project/core/layer.py:126: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the core instead.
  super(WeightedAspectEmb, self).__init__(**kwargs)


In [4]:
import torch

word_emb = inference_model.get_layer('word_embedding').get_weights()[0]
word_emb = torch.from_numpy(word_emb)

word_emb.shape

torch.Size([25055, 128])

In [5]:
aspect_embeddings = inference_model.get_layer('weighted_aspect_emb').W
vocab_inv = manager.embedding_model.model.wv.index_to_key

In [6]:
from core.evaluation import normalize_embedding_matrix

word_emb = normalize_embedding_matrix(word_emb)
aspect_embeddings = normalize_embedding_matrix(aspect_embeddings)

In [7]:
from core.evaluation import extract_top_k_words_of_aspect

aspects_top_k_words = [extract_top_k_words_of_aspect(a, word_emb, vocab_inv) for a in aspect_embeddings]


Given aspect most representative words are:
Word:  successor (0.739248514175415)
Word:  caylus (0.7310448288917542)
Word:  tigris (0.7292113304138184)
Word:  vein (0.7269777655601501)
Word:  concordia (0.72653728723526)
Word:  nova (0.7137109041213989)
Word:  brass (0.7124441862106323)
Word:  patchwork (0.7122059464454651)
Word:  goa (0.7093117237091064)
Word:  lorenzo (0.7025164365768433)
Word:  predecessor (0.6919888257980347)
Word:  scythe (0.6902366876602173)
Word:  carcassone (0.6799775958061218)
Word:  splendor (0.6772584915161133)
Word:  istanbul (0.677189826965332)
Word:  steroid (0.6756918430328369)
Word:  keyflower (0.674812376499176)
Word:  reimplementation (0.6687012910842896)
Word:  sequel (0.6677713394165039)
Word:  uwe (0.6667392253875732)
Word:  rosenberg (0.6656197309494019)
Word:  caverna (0.6651198863983154)
Word:  orleans (0.6585699915885925)
Word:  og (0.6574374437332153)
Word:  birmingham (0.6533355116844177)

Given aspect most representative words are:
Word:  il

![title](./../resources/umass.png)


### Dataset generation

In [8]:
import dataset

vocabulary = manager.embedding_model.model.wv.key_to_index
ds = dataset.PositiveNegativeCommentGeneratorDataset(
    vocabulary=vocabulary, csv_dataset_path="../data/corpus.preprocessed.kickstarter_removed.256k.csv", negative_size=15
)

Loading spacy model.
Loading dataset from file: ../data/corpus.preprocessed.kickstarter_removed.256k.csv
Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/200529 [00:00<?, ?it/s]

Max sequence length calculation in progress...
We loose information on 538 points.This is 0.2682903719661495% of the dataset.
Padding sequences to max length (256).
Max sequence length is:  1999  but we will limit sequences to 256 tokens.


In [10]:
from core.evaluation import coherence

print(manager.embedding_model.model.wv.index_to_key[13])
# todo pass indexwords
for i in range(len(aspects_top_k_words)):
    i_th_aspect_top_k = aspects_top_k_words[i][0:10]
    word_indexes = list(map(lambda x: x[2], i_th_aspect_top_k))
    print(word_indexes)
    print(coherence(word_indexes, ds))

rule
[3137, 1547, 2765, 2814, 1952, 2546, 1464, 2151, 4040, 3779]
-17.125883244267772
[1523, 1609, 3186, 1658, 1554, 1200, 3539, 2581, 145, 2327]


KeyboardInterrupt: 

4.4. Word2vec Coherence Score
[https://www.baeldung.com/cs/topic-modeling-coherence-score#4-word2vec-coherence-score